# DECISION TREE CLASSIFIER : C45

In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("dataset.csv")
df

,sno,cgpa,interact,pk,cs,jo
0,1,>=9,yes,v.good,good,yes
1,2,>=8,no,good,moderate,yes
2,3,>=9,no,average,poor,no
3,4,<8,no,average,good,no
4,5,>=8,yes,good,moderate,yes
5,6,>=9,yes,good,moderate,yes
6,7,<8,yes,good,poor,no
7,8,>=9,no,v.good,good,yes
8,9,>=8,yes,good,good,yes
9,10,>=8,yes,average,good,yes


In [3]:
def entropy_(df):
    entropy_=0
    yes=df[df['jo']=='yes'].value_counts().sum()
    no=df[df['jo']=='no'].value_counts().sum()
    entropy_=-(yes/len(df)*math.log2(yes/len(df)))-(no/len(df)*math.log2(no/len(df)))
    print(f'Entropy Info(Job offer) = {entropy_}\n')
    return entropy_

In [4]:
def entropy_info(df,attribute,entropy1):
    entropy_info=0
    values=df[attribute].value_counts()
    print(f'{values}\n')
    for value in values.index:
        yes=df[df['jo']=='yes'][df[attribute]==value].value_counts().sum()
        no=df[df['jo']=='no'][df[attribute]==value].value_counts().sum()
        print(f'For {attribute} {value}, Job offer : Yes = {yes} and No = {no}')
        if yes==0 or no==0:
            entropy_info+=0
        elif yes==no:
            entropy_info+=((yes+no)/len(df))*1
        else:
            entropy_info+=((yes+no)/len(df))*(-(yes/(yes+no)*math.log2(yes/(yes+no)))-(no/(yes+no)*math.log2(no/(yes+no))))
    gain=entropy1-entropy_info
    print(f'\nEntropy Info({attribute},job offer) = {entropy_info}')
    print(f'Gain ({attribute}) = {gain}')
    return gain

In [5]:
def split_info(df,attribute,gain):
    split_info=0
    total=df[attribute].value_counts().sum()
    values=df[attribute].value_counts()
    for value in values:
        split_info += -(value/total)*math.log2(value/total)
    gain_ratio = gain/split_info
    print(f'Split Info ({attribute}) = {split_info}')
    print(f'Gain Ratio ({attribute}) =  {gain_ratio}\n')
    return gain_ratio

In [6]:
def bestattribute(df):
    entropy = entropy_(df)
    attributes=['cgpa','interact','pk','cs']
    root=''
    maxgainratio=0
    for attribute in attributes:
        gain = entropy_info(df,attribute,entropy)
        gain_ratio=split_info(df,attribute,gain)
        if gain_ratio>maxgainratio:
            maxgainratio=gain_ratio
            root=attribute
    print(f'Since Gain ratio of attribute {root} is greater, Root attribute is {root}')
    return root

In [7]:
def splitdata(df,attribute):
    return df[attribute].unique()

In [8]:
def buildtree(i,df,tree=None):
    print(f'\nIteration {i+1}\n')
    attribute=bestattribute(df)
    if tree is None:
        tree={}
        tree[attribute]={}
    for value in splitdata(df,attribute):
        subtable=df[df[attribute]==value].reset_index(drop=True)
        clvalue,counts=np.unique(subtable['jo'],return_counts=True)
        if len(counts)==1:
            tree[attribute][value]=clvalue[0]
        else:
            tree[attribute][value]=buildtree(i+1,subtable)
    return tree

In [9]:
buildtree(0,df)


Iteration 1

Entropy Info(Job offer) = 0.8812908992306927

>=9    4
>=8    4
<8     2
Name: cgpa, dtype: int64

For cgpa >=9, Job offer : Yes = 3 and No = 1
For cgpa >=8, Job offer : Yes = 4 and No = 0
For cgpa <8, Job offer : Yes = 0 and No = 2

Entropy Info(cgpa,job offer) = 0.32451124978365314
Gain (cgpa) = 0.5567796494470396
Split Info (cgpa) = 1.5219280948873621
Gain Ratio (cgpa) =  0.36583834106055246

yes    6
no     4
Name: interact, dtype: int64

For interact yes, Job offer : Yes = 5 and No = 1
For interact no, Job offer : Yes = 2 and No = 2

Entropy Info(interact,job offer) = 0.7900134529890125
Gain (interact) = 0.09127744624168022
Split Info (interact) = 0.9709505944546686
Gain Ratio (interact) =  0.09400833241463322

good       5
average    3
v.good     2
Name: pk, dtype: int64

For pk good, Job offer : Yes = 4 and No = 1
For pk average, Job offer : Yes = 1 and No = 2
For pk v.good, Job offer : Yes = 2 and No = 0

Entropy Info(pk,job offer) = 0.636452797660028
Gain (pk) = 

{'cgpa': {'>=9': {'pk': {'v.good': 'yes', 'average': 'no', 'good': 'yes'}},
  '>=8': 'yes',
  '<8': 'no'}}